In [1]:
# public packages
import andes
import ams
import cvxpy as cp
import numpy as np
import gurobipy

# local packages
import anci_fun as af

# v1 try to include dynamic constraints using reduced input/output NN model

# ------ nn information ------
# input:   [M, D]
# output:  [rocof_max, fnadir, dtheta_max, eig_max]
# nn_diam: [16, 128, 4]

# Para preparetion

In [2]:
# neural network parameters 
nn_name = 'trained_model.pth'
nn_weights, nn_bias, nn_diam = af.extract_nn_para(nn_name) # three list

In [3]:
# MD parameters
vsg_num = 8

lb_M = np.array([0, 0, 0, 0, 0, 0, 0, 0])
ub_M = np.array([6, 6, 6, 6, 6, 6, 6, 6])

lb_D = np.array([0, 0, 0, 0, 0, 0, 0, 0])
ub_D = np.array([4, 4, 4, 4, 4, 4, 4, 4])

In [4]:
sf_load = 1.0

# y = [rocof_max, fnadir, dtheta_max, eig_max]
lb_y = np.array([-0.5, -0.6, -3.14, -10000])
ub_y = np.array([ 0.5,  0.6,  3.14,      0])

# Load ams model

In [5]:
ssp = ams.load('ams_case123_REGCV1_v5.xlsx')

ssp.LDOPF.run()

ssp.LDOPF.om.mdl.solve()

ssp.LDOPF.om.vars

LDOPF solved as optimal in 0.0598 seconds with exit code 0.


OrderedDict([('pg', Variable((11,))),
             ('pn', Variable((117,))),
             ('qg', Variable((11,))),
             ('qn', Variable((117,))),
             ('vsq', Variable((117,))),
             ('pl', Variable((116,))),
             ('ql', Variable((116,)))])

# Define new variables, constraints, and obj

In [6]:
# M and D

M = cp.Variable(vsg_num, name="M")
D = cp.Variable(vsg_num, name="D")

constraints_M = [M >= lb_M, M <= ub_M]
constraints_D = [D >= lb_D, D <= ub_D]

In [7]:
# data for the neural network linearization
pg    = ssp.LDOPF.om.pg

x1    = cp.hstack([M, D])  # this version doesn't incelu load scaling factor

w1    = nn_weights[0]
w2    = nn_weights[1]

b1    = nn_bias[0]
b2    = nn_bias[1]

hdown = -10
hup   = 10

In [8]:
# neural network linization

z = cp.Variable(nn_diam[1], name="z")
a = cp.Variable(nn_diam[1], name="a", boolean=True)

z_ = w1 @ x1 + b1

y = w2 @ z + b2

constraints_nn = [
                    # linization constraints
                    z <= z_ - hdown * (1 - a),
                    z >= z_,
                    z <= hup * a,
                    z >= 0,
                    # output constraints
                    # y <= ub_y,
                    y >= lb_y
                    # y[0] <= ub_y[0]                
                ]

In [9]:
y[0]

Expression(AFFINE, UNKNOWN, ())

In [10]:
# extract original ams constraints
constriants_ams = [ssp.LDOPF.om.constrs[cname] for cname in ssp.LDOPF.om.constrs.keys()]

In [11]:
constraints = constraints_M + constraints_D + constraints_nn + constriants_ams

# constraints = constriants_ams + constraints_M + constraints_D

# Formulate new problem

pg = ['PV_1',
 'PV_10',
 'PV_11',
 'PV_2',
 'PV_3',
 'PV_4',
 'PV_5',
 'PV_6',
 'PV_7',
 'PV_8',
 'PV_9']

 SG: PV_1, PV_2, PV_3 (slack bus)

 IBR: PV_4, PV_5, ..., PV_11

In [12]:
c2 = np.array([0.01, 0, 0, 0.01, 0.02, 0, 0, 0, 0, 0, 0])
c1 = np.array([  20, 0, 0,   20,   30, 0, 0, 0, 0, 0, 0])

d  = np.array([0, 0, 0, 0, 0, 0, 0, 0])

pg_power = pg**2

cost = cp.sum(c2 @ pg_power + c1 @ pg + d @ M)

obj = cp.Minimize(cost)

In [13]:
# Create the optimization problem
prob = cp.Problem(obj, constraints)

# Solve the optimization problem

# prob.solve(solver=cp.GUROBI, verbose=True)

prob.solve(solver=cp.ECOS_BB, verbose=True)

# prob.solve()

                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Sep 11 10:08:17 AM: Your problem has 877 variables, 22 constraints, and 0 parameters.
(CVXPY) Sep 11 10:08:17 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Sep 11 10:08:17 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Sep 11 10:08:17 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Sep 11 10:08:17 AM: Compiling problem (target solver=ECOS_BB).
(CVXPY) Sep 11 10:08:17 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffi

15  +2.000e+00  +2.000e+00  +2e-05  1e-16  2e-16  6e-12  1e-08  0.9739  1e-04   1  1  1 |  0  0
16  +2.000e+00  +2.000e+00  +4e-05  1e-16  5e-14  4e-12  3e-08  0.9890  9e-01   1  1  1 |  0  0
17  +2.000e+00  +2.000e+00  +2e-06  9e-18  2e-15  2e-13  1e-09  0.9591  2e-04   1  1  1 |  0  0
18  +2.000e+00  +2.000e+00  +1e-07  8e-18  2e-16  9e-15  9e-11  0.9890  2e-02   1  1  1 |  0  0
19  +2.000e+00  +2.000e+00  +5e-05  3e-15  9e-14  3e-14  3e-08  0.9890  1e+00   1  1  1 |  0  0
20  +2.000e+00  +2.000e+00  +5e-07  2e-17  1e-15  1e-15  4e-10  0.9890  1e-04   1  1  1 |  0  0
21  +2.000e+00  +2.000e+00  +1e-05  1e-11  3e-14  1e-13  9e-09  0.9482  9e-01   1  1  1 |  0  0
Unreliable search direction detected, recovering best iterate (15) and stopping.

Close to OPTIMAL (within feastol=1.5e-16, reltol=9.5e-06, abstol=1.9e-05).
Runtime: 0.096636 seconds.


ECOS 2.0.10 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k

In [ ]:
ssp.LDOPF.om.pg.value

In [ ]:
M.value

In [ ]:
D.value

# 